In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [2]:
import Data_ing
X_train,y_train,test_dat = Data_ing.load_and_ing_data()
#X_train = train.drop(['id','target'],axis=1)
#X_train.replace(-1,-99999)
#y_train = train['target']

TypeError: load_and_ing_data() missing 3 required positional arguments: 'train_x', 'train_y', and 'test_dat'

In [6]:
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from collections import Counter
"""
target_0 = train_y==0
target_1 = train_y==1
train_down_x = np.concatenate((train_x[target_1],train_x[target_1],train_x[target_1],
                               train_x[target_0][0:int(sum(target_1)*3)]))
train_down_y = np.concatenate((train_y[target_1],train_y[target_1],train_y[target_1],
                               train_y[target_0][0:int(sum(target_1)*3)]))
train_down_y = train_down_y.reshape([-1,1])
"""
X_real_test, y_real_test = [X_train[:3000],y_train[:3000]]
y_real_test = y_real_test.reshape([-1,1])
#X_train, X_test, y_train, y_test = train_test_split(X_50 , y_50, test_size=0.02)
#X_train, y_train = shuffle(train_down_x,train_down_y, random_state=0)
sm = SMOTE(random_state = 42)
X_train, y_train = sm.fit_sample(X_train[3000:],y_train[3000:])
print('smote Done')
X_train, y_train = shuffle(X_train,y_train, random_state=0)
print(Counter(y_train))
y_train = y_train.reshape([-1,1])

N_FEATURES=X_train.shape[1]
num_classes = 1

NameError: name 'X_train' is not defined

In [7]:
y_train[:20]

NameError: name 'y_train' is not defined

In [68]:
tf.reset_default_graph()
input_size = X_train.shape[1]

# ### Placeholder variables
x = tf.placeholder(tf.float32, [None, N_FEATURES],name="X-input")
y_true = tf.placeholder(tf.float32, [None,num_classes],name="y-input") ##[None, num_classes]
train_st = tf.placeholder(tf.bool, name = 'train_status')
# ### Helper-functions for creating new variables
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01), name = 'weight')

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]),name='bias')


# ### Helper-function for creating a new Fully-Connected Layer
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True,  # Use Rectified Linear Unit (ReLU)?
                 use_sig=False):
    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)
    elif use_sig:
        layer = tf.nn.sigmoid(layer)
    return layer


# ###  Layers
#hyperparameters
n_nodes_hl1 = 200 #input_size*20
n_nodes_hl2 = int(n_nodes_hl1/2)
n_nodes_hl3 = int(n_nodes_hl2/4)
n_nodes_hl4 = int(n_nodes_hl2/4)

beta = 0.001  # l2 regularization


keep_prob = tf.placeholder(tf.float32)

layer_h1 = new_fc_layer(input=x,
                         num_inputs=input_size,
                         num_outputs=n_nodes_hl1,
                         use_relu=True)
batch_norm_1 = tf.layers.batch_normalization(layer_h1,training=train_st)
drop_h1 = tf.nn.dropout(batch_norm_1, 0.5 + 0.5*keep_prob)

layer_h2 = new_fc_layer(input=drop_h1,
                         num_inputs=n_nodes_hl1,
                         num_outputs=n_nodes_hl2,
                         use_relu=True,
                         use_sig = False)
batch_norm_2 = tf.layers.batch_normalization(layer_h2,training=train_st)
drop_h2 = tf.nn.dropout(batch_norm_2, 0.75 + 0.25*keep_prob)

layer_h3 = new_fc_layer(input=layer_h2,
                         num_inputs=n_nodes_hl2,
                         num_outputs=n_nodes_hl3,
                         use_relu=True,
                         use_sig = False)
batch_norm_3 = tf.layers.batch_normalization(layer_h3,training=train_st)
drop_h3 = tf.nn.dropout(layer_h3, 0.85 + 0.15*keep_prob)

layer_h4 = new_fc_layer(input=drop_h3,
                         num_inputs=n_nodes_hl3,
                         num_outputs=n_nodes_hl4,
                         use_relu=True,
                         use_sig = False)
batch_norm_4 = tf.layers.batch_normalization(layer_h4,training=train_st)
drop_h4 = tf.nn.dropout(layer_h4, 0.9 + 0.1*keep_prob)

y_pred = new_fc_layer(input=drop_h4,
                         num_inputs=n_nodes_hl3,
                         num_outputs=num_classes,
                         use_relu=False,
                         use_sig = False)
y_pred_soft_max = tf.nn.sigmoid(y_pred)
#y_pred_cls = tf.argmax(y_pred, axis=1)



# ### Cost-function and Optimizer
learning_rate = tf.placeholder(tf.float32, shape=[])

#loss = tf.losses.mean_squared_error(y_true,y_pred_soft_max)
loss = tf.losses.sigmoid_cross_entropy(y_true,y_pred) #,weights= y_true*10
#loss = tf.square(y_pred_soft_max - y_true)/2

var_s   = tf.trainable_variables() 
lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in var_s
                    if 'bias' not in v.name ]) * beta
cost = tf.reduce_mean(loss)
#error = tf.square(y_pred - y_true)/2 
#cost = tf.reduce_sum(error + error* y_true*5)

optimizer = tf.train.AdamOptimizer().minimize(cost)  #  1e-4

# ### Performance measures
correct_prediction = tf.equal(tf.round(y_pred_soft_max), y_true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




session = tf.Session()
session.run(tf.global_variables_initializer())



INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.


In [69]:
from sklearn.metrics import precision_score
#feed_dict_test = {x: X_test, y_true: y_test, keep_prob: 1.0}  #.values.reshape([-1,1])
feed_dict_real_test = {x: X_real_test, y_true:y_real_test, keep_prob: 1.0,train_st:False}
####
from sklearn.metrics import roc_auc_score
def gini(y_actual, y_pred):
  return 2*roc_auc_score(y_actual, y_pred)-1
####
def print_accuracy( r_valu = False, feed = feed_dict_real_test):
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed)
    #print(precision_score(y_real_test,pred))
    if r_valu:
        return acc
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

print_accuracy()
from eval import normalized_gini
def print_gini():
    y_p,y = session.run([y_pred_soft_max,y_true], feed_dict=feed_dict_real_test)
    #print(y[:,1])
    return gini(y,y_p)
print(print_gini())
    
saver = tf.train.Saver()  #search tf.saver.tensorflow
best_validation_accuracy = 0
best_validation_gini = 0

Accuracy on test-set: 3.8%
0.0200686016963


In [1]:
batch_size = 128
def optimize( hm_epochs = 10, learning_r = 1e-4):
    global best_validation_accuracy,best_validation_gini
    global global_step
    for epoch in range(hm_epochs):
            epoch_loss = 0
            i = 0
            len_train = X_train.shape[0]
            while i < len_train:
                start = i
                end = i + batch_size

                batch_x = np.array(X_train[start:end])
                batch_y = np.array(y_train[start:end])    #.values.reshape([-1,1]))
                
                feed_dict_train = {x: batch_x,
                                   y_true: batch_y,
                                   keep_prob: 0.0,
                                   learning_rate:learning_r,
                                   train_st:True}
                
                _, c = session.run([optimizer,cost], feed_dict=feed_dict_train)
                epoch_loss += c

                i += batch_size

                if i % 50000 == 0  or i == (len_train - 1):


                    acc_ = print_accuracy(True)
                    if acc_ > best_validation_accuracy:
                        best_validation_accuracy = acc_
                        saver.save(session, 'models/my_model_acc')
                        imp_str_acc = '*'*4
                    else:
                        imp_str_acc = ''
                    gini = print_gini()
                    if gini > best_validation_gini:
                        best_validation_gini = gini
                        saver.save(session, 'models/my_model_gini')
                        imp_str_gini = '*'*4
                    else:
                        imp_str_gini = ''

                    print('epoch:{} loss: {:.4f},  acc: {:.4f}{} __gini: {:.4f}{}'.format(epoch,epoch_loss,acc_*100,imp_str_acc,gini,imp_str_gini))
                    epoch_loss = 0



    print('Epoch {} completed out of {}'.format(epoch+1,hm_epochs))

    print('Optimization ended '+ 10*'_' + 'best accuracy: {}'.format(best_validation_accuracy))

In [2]:
#best_validation_accuracy = 0.5
#best_validation_gini= 0

In [3]:
optimize(50,1e-5)

NameError: name 'X_train' is not defined

In [63]:
session.run(y_pred_soft_max,feed_dict_real_test)

array([[  3.07041802e-07],
       [  8.84188651e-08],
       [  4.14646564e-07],
       ..., 
       [  2.80290891e-07],
       [  1.87435290e-07],
       [  6.19631749e-07]], dtype=float32)

In [40]:
#

#X_sub = test.drop(['id'],axis=1)
#X_sub.replace(-1,-99999)
#X_sub = prepare_data(test,['id'])
X_sub = test_dat

saver.restore(session,"models/my_model_acc")
sub1 = session.run(y_pred_soft_max,feed_dict={x:X_sub[:int(892816/2)],keep_prob:1.0,train:False})
sub2 = session.run(y_pred_soft_max,feed_dict={x:X_sub[int(892816/2):],keep_prob:1.0,train:False})
sub_tot_A = np.append(sub1,sub2)

INFO:tensorflow:Restoring parameters from models/my_model_acc


In [41]:
saver.restore(session,"models/my_model_gini")
sub1 = session.run(y_pred_soft_max,feed_dict={x:X_sub[:int(892816/2)],keep_prob:1.0,train:False})
sub2 = session.run(y_pred_soft_max,feed_dict={x:X_sub[int(892816/2):],keep_prob:1.0,train:False})
sub_tot_B = np.append(sub1,sub2)

INFO:tensorflow:Restoring parameters from models/my_model_gini


In [42]:
#test = pd.read_csv("data/test.csv")
test_id = test.id.values

sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = (sub_tot_A + sub_tot_B)/2

#series = pd.Series(sub_tot)
sub.to_csv('sub/submission_{}.csv'.format(int(best_validation_accuracy*100)) ,index = False)

In [45]:
sub_tot_B[sub_tot_B>0.5][:10]

array([ 0.56610572,  0.52855325,  0.58747709,  0.63089669,  0.53465188,
        0.58455157,  0.56880814,  0.66447335,  0.55966204,  0.54320621], dtype=float32)

In [46]:
sub_tot_A[sub_tot_B>0.5][:10]

array([ 0.4014574 ,  0.3196542 ,  0.12273712,  0.42307267,  0.34194165,
        0.43101513,  0.4848755 ,  0.31742668,  0.29546708,  0.32031181], dtype=float32)